# Import data

In [ ]:
import pandas as pd
import os
import missingno as msno
from IPython.display import display


file_path='remeha_export.parquet'


# usually, two decimals suffice for displaying DataFrames (NB internally, precision may be higher)
pd.options.display.precision = 2

import sys
sys.path.append('../data/')
sys.path.append('../view/')

%load_ext autoreload


%matplotlib inline
%matplotlib widget
import pylab as plt
import itertools
from plotter import Plot

In [ ]:
%%time
# Attempt to read the Parquet file
try:
    df = pd.read_parquet(
        file_path, 
        engine='pyarrow',
        use_nullable_dtypes=True
        )
    print("File was successfully read without specifying compression codec.")
except Exception as e:
    print(f"Error reading file: {e}")


In [ ]:
## Initial exploration: size, ids, start & stop times per id

In [ ]:
df.info()

In [ ]:
df.memory_usage()

In [ ]:
# Get the names of the levels in the MultiIndex
level_names = df.index.names

# Get the datatypes of the levels in the MultiIndex
level_dtypes = [df.index.get_level_values(level).dtype for level in range(df.index.nlevels)]

# Display the names and datatypes of the levels in the MultiIndex
for i, name in enumerate(level_names):
    print(f"Level {i}: Name = {name}, Dtype = {level_dtypes[i]}")

In [ ]:
df

In [ ]:
df['value'].count()

In [ ]:
df['value'].notna().sum()

In [ ]:
len(df)

In [ ]:
# deduplicate the measurements
df = df.reset_index().drop_duplicates().set_index(['id', 'source_category', 'source_type', 'timestamp', 'property']).sort_index()

In [ ]:
len(df)

In [ ]:
len(df.index.get_level_values('id').unique())

In [ ]:
df.groupby('id').size()

In [ ]:
df.reset_index().groupby(['id', 'source_type'])['timestamp'].agg(['min', 'max'])

In [ ]:
print('\n'.join(df.index.get_level_values('source_category').unique()))

In [ ]:
print('\n'.join(df.index.get_level_values('source_type').unique()))

In [ ]:
print('\n'.join(df.index.get_level_values('property').unique()))

In [ ]:
df.groupby(['source_category', 'source_type', 'property']).size()[counts != 0]

# Get overview

In [ ]:
Plot.nfh_measurements_plot(df, ids = [412715])
# Plot.nfh_measurements_plot(df, source_types = ['enelogic'], ids = [404873])
# Plot.nfh_measurements_plot(df, source_types = ['enelogic'])


In [ ]:
Plot.nfh_measurements_plot(df, properties = ['temp_in__degC'])

In [ ]:
df_unstacked = df.unstack(level='property')

In [ ]:
df_unstacked

In [ ]:
df_unstacked.info()

In [ ]:
df_unstacked.describe()

In [ ]:
for id in df_unstacked.index.get_level_values('id').unique():
    print(f"Description for id {id}:")
    display(df_unstacked[df_unstacked.index.get_level_values('id') == id].describe())


# Detect stuck values

In [ ]:
# Get unique ids
unique_ids = df_unstacked.index.get_level_values('id').unique()

# Initialize an empty DataFrame to store the variance per id per column
variance_df = pd.DataFrame(index=unique_ids, columns=df_unstacked.columns)

# Calculate variance for each id and each column
for id in unique_ids:
    id_df = df_unstacked[df_unstacked.index.get_level_values('id') == id]
    variance_df.loc[id] = id_df.var()

# Define a function to apply conditional formatting
def highlight_zero_variance(val):
    if pd.notna(val) and val == 0:
        return 'background-color: yellow'
    return ''

# Apply conditional formatting to the DataFrame
variance_df_styled = variance_df.style.applymap(highlight_zero_variance)

# Display the styled DataFrame
variance_df_styled


In [ ]:
for id in df_unstacked.index.get_level_values('id').unique():
    df_unstacked[df_unstacked.index.get_level_values('id') == id].describe()

In [ ]:
msno.bar(df_unstacked)

In [ ]:
msno.matrix(df_unstacked)

In [ ]:
msno.matrix(df_unstacked.swaplevel('timestamp', 'id', axis=0))

In [ ]:
for id in df_unstacked.index.get_level_values('id').unique():
    msno.matrix(df_unstacked[df_unstacked.index.get_level_values('id') == id])